In [ ]:
!pip install EbookLib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for EbookLib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=7ddd05b1cd793f4cad7a372b44b2b553697d4c2608aa17e683fa78c6018e141a
  Stored in directory: /root/.cache/pip/wheels/0f/38/cc/a3728bb72a315d9d8766fb71d362136372066fc25ad838f8fa
Successfully built EbookLib


In [ ]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Path to EPUB file in Google Drive
#epub_path = '/content/drive/My Drive/TORI_dataset/Gatsby.epub'
#epub_path = '/content/drive/My Drive/TORI_dataset/blue_castle.epub'
#epub_path = '/content/drive/My Drive/TORI_dataset/little_woman.epub'
#epub_path = '/content/drive/My Drive/TORI_dataset/alice_wonderland.epub'
epub_path = '/content/drive/My Drive/TORI_dataset/enchanted_april.epub'
#epub_path = '/content/drive/My Drive/TORI_dataset/A_roomtour.epub'

# Read the EPUB file
book = epub.read_epub(epub_path)

In [ ]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import re


# Function to validate the EPUB structure
def is_valid(epub_book):
    """Validates the structure of an EPUB book."""
    if not epub_book.toc:
        print('Table of content is missing')
        return False
    if not epub_book.spine:
        print('Spine is missing')
        return False

    # Check for stylesheets
    stylesheets = [item for item in epub_book.get_items_of_type(ebooklib.ITEM_STYLE)]
    if len(stylesheets) == 0:
        print('No stylesheets')
        return False

    # Ensure chapters are not empty
    for item in epub_book.spine:
        if isinstance(item, epub.EpubHtml):
            if not item.content.strip():
                print(f"Invalid book: Chapter '{item.get_id()}' is empty.")
                return False

    return True

# Validate the book
if is_valid(book):
    print("The EPUB book is valid.")
else:
    print("The EPUB book is not valid.")


The EPUB book is valid.


In [ ]:
# Function to convert a chapter object into a string, filtering out non-textual elements
def chapter_to_str(chapter):
    """Converts a chapter object into a string, filtering out non-textual elements."""
    soup = BeautifulSoup(chapter.get_body_content(), 'html.parser')
    # Filtering out script and style elements
    for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
    text = ' '.join(para.get_text(strip=True) for para in soup.find_all('p'))
    return text

# Function to determine if the item is a Gutenberg cover wrapper
def is_wrapper(id):
    """Determine if the item is the Gutenberg cover wrapper."""
    patterns = [r'coverpage', r'wrapper']
    return any(re.search(pattern, id, re.IGNORECASE) for pattern in patterns)


In [ ]:
def is_gutenberg_intro(text):
    """Checks if text contains common Gutenberg intro phrases."""
    patterns = [r'Project Gutenberg', r'\bEBook\b', r'\blicense\b', r'This eBook is for the use of']
    return any(re.search(pattern, text, re.IGNORECASE) for pattern in patterns)

In [ ]:
# Enhanced function to dynamically extract descriptions, including handling pronouns
def enhanced_extract_descriptions(text, characters, appearance_words):
    sentences = text.split('.')  # Split the text into sentences
    character_descriptions = {char: [] for char in characters}
    last_mentioned_character = None

    for sentence in sentences:
        words = sentence.split()
        for i, word in enumerate(words):
            # Check if the word is a character name
            for char in characters:
                if char in sentence:
                    last_mentioned_character = char

            # Handle coreference with pronouns
            if word.lower() in ['he', 'she', 'his', 'her', 'him', 'herself', 'himself'] and last_mentioned_character:
                if i + 1 < len(words) and words[i + 1] in appearance_words:
                    phrase = f"{words[i]} {words[i + 1]}"
                    character_descriptions[last_mentioned_character].append(phrase)

            # Handle direct descriptions (like "long hair")
            if word in appearance_words and i > 0:
                previous_word = words[i-1]
                if previous_word.isalpha() and previous_word not in appearance_words:
                    phrase = f"{previous_word} {word}"
                    if last_mentioned_character:
                        character_descriptions[last_mentioned_character].append(phrase)

    return character_descriptions

In [ ]:
# Improved filtering function to remove generic phrases
def improved_filter_phrases(character_descriptions):
    filtered_descriptions = {}
    for character, phrases in character_descriptions.items():
        filtered_phrases = []
        for phrase in phrases:
            words = phrase.split()
            # Remove phrases that start with pronouns and are too generic
            if len(words) > 1 and not re.match(r'\b(her|his|their|your|my|our)\b', words[0]):
                filtered_phrases.append(phrase)
        if filtered_phrases:
            filtered_descriptions[character] = filtered_phrases
    return filtered_descriptions

In [ ]:
# Function to filter out incorrect gender pronouns for each character
def correct_gender_pronouns(character_descriptions):
    corrected_descriptions = {}
    for character, phrases in character_descriptions.items():
        if character == ["Fisher","Wilkins"]:
            filtered_phrases = [phrase for phrase in phrases if not re.match(r'\bhis\b', phrase)]
        elif character in ["Wilkins"]:
            filtered_phrases = [phrase for phrase in phrases if not re.match(r'\bher\b', phrase)]
        else:
            filtered_phrases = phrases
        if filtered_phrases:
            corrected_descriptions[character] = filtered_phrases
    return corrected_descriptions

In [ ]:
# Process each chapter in the book
def process_chapters(book):
    chapters_with_descriptions = {}
    for spine_item in book.spine:
        item_id = spine_item[0] if isinstance(spine_item, tuple) else spine_item
        item = book.get_item_with_id(item_id)

        if isinstance(item, epub.EpubHtml) and not is_wrapper(item.get_id()):
            chapter_text = chapter_to_str(item)
            if not is_gutenberg_intro(chapter_text):  # Skip Gutenberg intro parts
                chapter_title = f"Chapter {item.get_id()}"
                character_descriptions = enhanced_extract_descriptions(chapter_text, characters_of_interest, appearance_words)
                filtered_descriptions = improved_filter_phrases(character_descriptions)
                gender_corrected_descriptions = correct_gender_pronouns(filtered_descriptions)
                chapters_with_descriptions[chapter_title] = gender_corrected_descriptions
    return chapters_with_descriptions

In [ ]:
#Load and parse the relationships.txt file
appearance_words = set()

# Path to the relationships.txt file
relationships_path = '/content/drive/My Drive/TORI_wordnet/enchanted_april/relationships.txt'

# Read and process the file
with open(relationships_path, 'r') as file:
    content = file.read()

# Use regular expressions to find all appearance-related terms
matches = re.findall(r'synonyms:\s*\[(.*?)\]', content)

# Process and clean up the words
for match in matches:
    words = re.findall(r"'(.*?)'", match)
    appearance_words.update(words)

# Convert the set to a sorted list for consistency
appearance_words = sorted(list(appearance_words))

print("Appearance-related words:", appearance_words)


Appearance-related words: ['affection', 'affectionateness', 'anatomy', 'animal foot', 'arm', 'arms', 'aspect', 'auricle', 'back', 'back talk', 'back up', 'backbone', 'backrest', 'backtalk', 'backward', 'backwards', 'base', 'bet on', 'binding', 'blazon', 'blazonry', 'bod', 'boldness', 'book binding', 'bosom', 'brain', 'branch', 'brass', 'bridge player', 'brow', 'build', 'build up', 'buttock', 'calculate', 'capitulum', 'caput', 'case', 'center', 'centre', 'channelise', 'channelize', 'chassis', 'cheek', 'chief', 'cipher', 'coat of arms', 'compute', 'confront', 'core', 'count on', 'cover', 'custody', 'cypher', 'deal', 'design', 'digit', 'direct', 'dorsum', 'drumhead', 'ear', 'endorse', 'enter', 'envision', 'essence', 'estimate', 'expression', 'eye', 'eyeball', 'eyes', 'face', 'face up', 'facial expression', 'fancy', 'fig', 'figure', 'figure of speech', 'flesh', 'fondness', 'font', 'foot', 'forecast', 'forefront', 'forehead', 'foreland', 'form', 'fortify', 'foundation', 'fount', 'fountainhe

In [ ]:
characters_of_interest = ["Fisher","Wilkins","Wilkins"]

In [ ]:
final_chapters_with_enhanced_descriptions = process_chapters(book)

In [ ]:
# Save the final organized data to a text file
#output_path_final_enhanced = '/content/drive/My Drive/Coref/blue_castle_organized_descriptions_by_chapter.txt'
#output_path_final_enhanced = '/content/drive/My Drive/Coref/gatsby_organized_descriptions_by_chapter.txt'
#output_path_final_enhanced = '/content/drive/My Drive/Coref/little_woamn_organized_descriptions_by_chapter.txt'
#output_path_final_enhanced = '/content/drive/My Drive/Coref/alice_wonderland_organized_descriptions_by_chapter.txt'
output_path_final_enhanced = '/content/drive/My Drive/Coref/april_organized_descriptions_by_chapter.txt'
#output_path_final_enhanced = '/content/drive/My Drive/Coref/roomtour_organized_descriptions_by_chapter.txt'


with open(output_path_final_enhanced, 'w') as file:
    for chapter, characters in final_chapters_with_enhanced_descriptions.items():
        file.write(f"{chapter}:\n")
        for character, descriptions in characters.items():
            file.write(f"  {character}:\n")
            for description in descriptions:
                file.write(f"    - {description}\n")
        file.write("\n")